In [1]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
env_name="CartPole-v1"
env = gym.make(env_name)

In [3]:
class PolicyNet(nn.Module):
    def __init__(self, env) -> None:
        super().__init__()
        self.action_dim = env.action_space.n
        self.net = nn.Sequential(
            nn.Linear(4, 100),
            nn.LeakyReLU(),
            nn.Dropout(0.6),
            nn.Linear(100, self.action_dim),
            nn.Softmax(dim=-1)
        )
    def forward(self, state):
        state = torch.tensor(state, dtype=torch.float)
        return self.net(state)

In [7]:
## Casual Ablation:
## 1. The use of torch.distribution helped the gradient calculation
## 2. Use of Adam instead of SGD helped with the learning

def grad_descent(actions, rewards):
    G = []
    R = 0
    for r in rewards[::-1]:
        R = r + gamma * R
        G.insert(0, R)
    for i in range(len(G)):
        G[i] *= -actions[i]
    optimizer.zero_grad()
    loss = torch.cat(G).sum()
    loss.backward()
    optimizer.step()

def rand_action():
    return np.random.choice(range(2), p=[0.5, 0.5])

# def sample_action(prob):
#     dist = Categorical(prob)
#     action = dist.sample()
#     log_probs.append(dist.log_prob(action))
#     return action.item()

def sample_action(prob):
    action = np.random.choice(range(pol_net.action_dim), p=prob.squeeze(0).detach().numpy())
    log_probs.append(torch.log(prob[0][action]).unsqueeze(0))
    return action

num_epi = 300
pol_net = PolicyNet(env)

gamma = 0.99
optimizer = optim.Adam(pol_net.parameters(), lr = 1e-3)
log_freq = 10



for i in range(num_epi):
    done = False
    state = env.reset()
    log_probs, rewards = [], []
    for t in range(20000):
        old_state = state
        action_prob = pol_net(torch.tensor(state).unsqueeze(0))
        action = sample_action(action_prob)
        state, reward, done, info = env.step(action)
        rewards.append(reward)
        if done:
            grad_descent(log_probs, rewards)
            break
    if i % log_freq == 0:
        print('Total Reward: %d' % sum(rewards))

/var/folders/s6/qk66mxfn1jqgsjslnssw3mjm0000gn/T/ipykernel_93496/155788214.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state, dtype=torch.float)


RuntimeError: zero-dimensional tensor (at position 0) cannot be concatenated

In [ ]:
state = env.reset()
for t in range(20000):
    env.render()
    old_state = state
    action_prob = pol_net(torch.tensor(state).unsqueeze(0))
    action = sample_action(action_prob)
    state, reward, done, info = env.step(action)
    if done:
        break
env.close()